In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from itertools import product
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from copy import deepcopy
import time
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

#import wandb
#wandb.login(key='0057f517a0acb0992a4695e1d89cff691b8a8960')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
SABR_price = pd.read_csv('/kaggle/input/sabr-def1/SABR_model_DEF_PARTE01.csv')
SABR_price["S0"] = SABR_price["S0"] * np.exp(SABR_price["r"] * SABR_price["T"])
SABR_price.head()

,S0,alpha,beta,rho,nu,r,T,K,IV,IV_Approx,BS Price,SABR Price,BS Price IV
0,85.495288,0.067447,1.0,0.520296,0.124547,0.024805,0.2,50,0.386060,0.060740,35.742719,35.745352,35.745352
1,85.495288,0.067447,1.0,0.520296,0.124547,0.024805,0.2,55,0.321857,0.060176,30.767462,30.770095,30.770095
2,85.495288,0.067447,1.0,0.520296,0.124547,0.024805,0.2,60,0.262809,0.060290,25.792205,25.794838,25.794838
3,85.495288,0.067447,1.0,0.520296,0.124547,0.024805,0.2,65,0.207931,0.060993,20.816948,20.819582,20.819582
4,85.495288,0.067447,1.0,0.520296,0.124547,0.024805,0.2,70,0.156389,0.062161,15.841691,15.844325,15.844325


In [8]:
class OptionPricingDatasetSABR(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32).to(device)
        self.y = torch.tensor(y.values, dtype=torch.float32).to(device)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [7]:
X_sabr = SABR_price.drop(['BS Price', 'BS Price IV','IV','SABR Price','beta','IV_Approx'], axis=1)  # Features
y_sabr = SABR_price['SABR Price']  # Target

X_train_sabr , X_temp_sabr , y_train_sabr , y_temp_sabr  = train_test_split(X_sabr , y_sabr , test_size=0.2, random_state=42)         # Train/Test+Val
X_val_sabr , X_test_sabr , y_val_sabr , y_test_sabr  = train_test_split(X_temp_sabr, y_temp_sabr, test_size=0.5, random_state=42)  # Val/Test


scaler = MinMaxScaler()
X_train_normalized_sabr  = scaler.fit_transform(X_train_sabr)  # Fit on training data and transform
X_val_normalized_sabr  = scaler.transform(X_val_sabr)         # Transform validation data
X_test_normalized_sabr  = scaler.transform(X_test_sabr)       # Transform test data

# Convert normalized data back into DataFrames for DataLoader compatibility
X_train_normalized_sabr  = pd.DataFrame(X_train_normalized_sabr , columns=X_sabr.columns)
X_val_normalized_sabr  = pd.DataFrame(X_val_normalized_sabr , columns=X_sabr.columns)
X_test_normalized_sabr  = pd.DataFrame(X_test_normalized_sabr , columns=X_sabr.columns)

# Create PyTorch datasets
train_dataset_sabr  = OptionPricingDatasetSABR(X_train_normalized_sabr , y_train_sabr)
val_dataset_sabr  = OptionPricingDatasetSABR(X_val_normalized_sabr , y_val_sabr)
test_dataset_sabr  = OptionPricingDatasetSABR(X_test_normalized_sabr , y_test_sabr)

# Create DataLoaders
#train_loader_sabr  = torch.utils.data.DataLoader(train_dataset_sabr , batch_size=256, shuffle=True)
#val_loader_sabr  = torch.utils.data.DataLoader(val_dataset_sabr , batch_size=256, shuffle=False)
#test_loader_sabr  = torch.utils.data.DataLoader(test_dataset_sabr , batch_size=256, shuffle=False)

In [21]:
def train_model(model, train_loader, val_loader, mse_criterion, optimizer, device, num_epochs=50, patience=5):
    best_loss = float('inf')
    counter = 0
    best_model = deepcopy(model.state_dict())
    all_predictions = []
    all_targets = []
    for epoch in range(num_epochs):
        model.train()
        running_loss_mse = 0.0

        for inputs, targets in tqdm(train_loader, desc="Training Epoch", unit="batch"):
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(inputs)
            targets = targets.unsqueeze(1)

            # Compute MSE loss for monitoring (not used in training)
            loss_mse = mse_criterion(outputs, targets)

            # Backward pass and optimization
            loss_mse.backward()
            optimizer.step()

            running_loss_mse += loss_mse.item()

            # Collect predictions and true values for MAE & R² computation
            all_predictions.extend(outputs.cpu().detach().numpy().flatten())
            all_targets.extend(targets.cpu().detach().numpy().flatten())
            
        avg_train_loss_mse = running_loss_mse / X_train_sabr.shape[0]
        avg_train_loss_mae = mean_absolute_error(all_targets, all_predictions)
        train_r2_score = r2_score(all_targets, all_predictions)

        print(f"Epoch {epoch + 1}/{num_epochs} - "
              f"Train MSE: {avg_train_loss_mse:.10f}, "
              f"Train MAE: {avg_train_loss_mae:.10f}, "
              f"Train R²: {train_r2_score:.10f}")

        #print(f"Epoch {epoch + 1}/{num_epochs},  Train MSE Loss: {avg_train_loss_mse:.10f}")

        # Validation after each epoch
        avg_val_loss_mse = validation_model(model, val_loader, mse_criterion, device)


        # Early stopping based on custom loss
        if avg_val_loss_mse < best_loss:
            best_loss = avg_val_loss_mse
            counter = 0
            best_model = deepcopy(model.state_dict())  # Save the best model state
        else:
            counter += 1

        if counter >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break

    model.load_state_dict(best_model)  # Restore the best model
    return model, avg_train_loss_mse, avg_val_loss_mse, best_loss

# Validation function
def validation_model(model, val_loader,  mse_criterion, device):
    model.eval()
    val_loss_mse = 0.0
    all_predictions = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in tqdm(val_loader, desc="Validation", unit="batch"):
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = model(inputs)
            targets = targets.unsqueeze(1)

            loss_mse = mse_criterion(outputs, targets)        # MSE loss

            val_loss_mse += loss_mse.item()
            
            # Collect predictions and true values for MAE & R² computation
            all_predictions.extend(outputs.cpu().detach().numpy().flatten())
            all_targets.extend(targets.cpu().detach().numpy().flatten())
            
        avg_val_loss_mse = val_loss_mse / X_val_sabr.shape[0]
        avg_val_loss_mae = mean_absolute_error(all_targets, all_predictions)
        val_r2_score = r2_score(all_targets, all_predictions)

        print(f"Val MSE: {avg_val_loss_mse:.10f}, "
              f"Val  MAE: {avg_val_loss_mae:.10f}, "
              f"Val  R²: {val_r2_score:.10f}")
    #print(f" Validation MSE Loss: {avg_val_loss_mse:.10f}")
    return  avg_val_loss_mse

# Testing function
def test_model(model, test_loader, mse_criterion, device):
    model.eval()
    
    test_loss_mse = 0.0
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for inputs, targets in tqdm(test_loader, desc="Testing", unit="batch"):
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = model(inputs)
            targets = targets.unsqueeze(1)

            loss_mse = mse_criterion(outputs, targets)        # MSE loss

            test_loss_mse += loss_mse.item()

            all_predictions.extend(outputs.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    avg_test_loss_mse = test_loss_mse / X_test_sabr.shape[0]
    avg_test_loss_mae = mean_absolute_error(all_targets, all_predictions)
    test_r2_score = r2_score(all_targets, all_predictions)

    print(f"Test MSE: {avg_test_loss_mse:.10f}, "
          f"Test MAE: {avg_test_loss_mae:.10f}, "
          f"Test R²: {test_r2_score:.10f}")

    return  avg_test_loss_mse, all_predictions, all_targets

In [22]:
def objective(trial):
    # Define hyperparameters to optimize
    num_layers = trial.suggest_int("num_layers", 2, 5)  # Number of hidden layers
    hidden_size = trial.suggest_int("hidden_size", 50, 500)  # Number of neurons per layer
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.2)  # Dropout rate
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-3, log=True)  # Learning rate
    batch_size = trial.suggest_categorical("batch_size", [32,64,128,256])  # Batch size

    # Define the MLP model dynamically
    class MLP(nn.Module):
        def __init__(self, input_dim, num_layers, hidden_size, dropout_rate):
            super(MLP, self).__init__()
            layers = [nn.Linear(input_dim, hidden_size), nn.ReLU(), nn.Dropout(dropout_rate)]
            for _ in range(num_layers - 2):
                layers.extend([nn.Linear(hidden_size, hidden_size), nn.ReLU(), nn.Dropout(dropout_rate)])
            layers.extend([nn.Linear(hidden_size, int(hidden_size/4)), nn.ReLU(), nn.Dropout(dropout_rate)])
            layers.extend([nn.Linear(int(hidden_size/4), int(hidden_size/4)), nn.ReLU(), nn.Dropout(dropout_rate)])    
            layers.append(nn.Linear(int(hidden_size/4), 1))  # Output layer
            self.model = nn.Sequential(*layers)

        def forward(self, x):
            return self.model(x)

    # Initialize model
    input_dim = X_train_sabr.shape[1]
    model = MLP(input_dim, num_layers, hidden_size, dropout_rate).to(device)

    # Define loss function and optimizer
    mse_criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoaders with the suggested batch_size
    train_loader_sabr = torch.utils.data.DataLoader(train_dataset_sabr, batch_size=batch_size, shuffle=True)
    val_loader_sabr = torch.utils.data.DataLoader(val_dataset_sabr, batch_size=batch_size, shuffle=False)

    # Train the model
    trained_model, _, _, best_val_loss = train_model(
        model, train_loader_sabr, val_loader_sabr, mse_criterion, optimizer, device, num_epochs=50, patience=5
    )

    return best_val_loss  # Minimize validation loss

# Run Optuna study with the updated objective function
study = optuna.create_study(study_name="MLP_Optimization", direction="minimize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=1)  # Try 30 different configurations

# Print best hyperparameters
print("Best Hyperparameters:", study.best_params)

[I 2025-03-07 18:10:16,601] A new study created in memory with name: MLP_Optimization
Training Epoch: 100%|██████████| 48765/48765 [02:23<00:00, 339.34batch/s]


Epoch 1/50 - Train MSE: 0.0496824509, Train MAE: 0.5428456068, Train R²: 0.9906111557


Validation: 100%|██████████| 6096/6096 [00:06<00:00, 934.89batch/s]


Val MSE: 0.0006512187, Val  MAE: 0.1427313536, Val  R²: 0.9998766053


Training Epoch: 100%|██████████| 48765/48765 [02:24<00:00, 336.61batch/s]


...

Epoch 6/50 - Train MSE: 0.0027030204, Train MAE: 0.3143320084, Train R²: 0.9979301993


Validation: 100%|██████████| 6096/6096 [00:06<00:00, 919.47batch/s]


Val MSE: 0.0063514960, Val  MAE: 0.4427323043, Val  R²: 0.9987965151
Early stopping triggered at epoch 6


[I 2025-03-07 18:25:43,502] Trial 0 finished with value: 0.0006512187028031105 and parameters: {'num_layers': 4, 'hidden_size': 225, 'dropout_rate': 0.006211959466474126, 'learning_rate': 0.00011768237318338051, 'batch_size': 64}. Best is trial 0 with value: 0.0006512187028031105.


Best Hyperparameters: {'num_layers': 4, 'hidden_size': 225, 'dropout_rate': 0.006211959466474126, 'learning_rate': 0.00011768237318338051, 'batch_size': 64}


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 200),  # Input layer
            nn.ReLU(),
            nn.Linear(200, 200),        # Hidden layer
            nn.ReLU(),
            nn.Linear(200, 200),         # Hidden layer
            nn.ReLU(),
            nn.Linear(200, 50),          # Hidden layer
            nn.ReLU(),
            nn.Linear(50, 1),          # Hidden layer (Matches checkpoint)
        )
    def forward(self, x):
        return self.model(x)

In [ ]:
# Training phase
trained_model, train_losses, val_loss, best_loss = train_model(
    model, train_loader_sabr, val_loader_sabr, mse_criterion, optimizer, device, num_epochs=50, patience=5
)

# Test phase
test_loss, predictions, true_values = test_model(trained_model, test_loader_sabr, mse_criterion, device)

# Save model
torch.save(trained_model.state_dict(), "mlp_SABR.pth")
print("Model saved!")